<a href="https://colab.research.google.com/github/suchirkolli/MyProjects/blob/main/AI_Doctor_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall accelerate peft bitsandbytes transformers trl gcsfs fsspec -y

!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0 \
fsspec==2024.12.0 gcsfs==2024.12.0


Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5
Found existing installation: transformers 4.51.2
Uninstalling transformers-4.51.2:
  Successfully uninstalled transformers-4.51.2
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
Found existing installation: gcsfs 2024.12.0
Uninstalling gcsfs-2024.12.0:
  Successfully uninstalled gcsfs-2024.12.0
Found existing installation: fsspec 2024.12.0
Uninstalling fsspec-2024.12.0:
  Successfully uninstalled fsspec-2024.12.0
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.45.5-py3-none-

In [2]:
!pip install huggingface_hub

In [3]:

import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

In [4]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
llama_model.gradient_checkpointing_enable()
llama_model.enable_input_require_grads()

In [6]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

In [7]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 2, max_steps = 50)

In [8]:
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=load_dataset(path="aboonaji/wiki_medical_terms_llam2_format", split="train"),
    peft_config=LoraConfig(task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1),
    processing_class=llama_tokenizer,
    formatting_func=lambda x: x["text"]
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
llama_sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: suchirkolli (suchirkolli-university-of-florida) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Step,Training Loss


In [13]:
#Chatting with bot
user_prompt = "Please tell me about Bursitis"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 500)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Please tell me about Bursitis [/INST]  Bursitis is a condition that occurs when the bursae, small fluid-filled sacs that cushion and reduce friction between muscles, tendons, and bones, become inflamed or irritated. hopefully, this information will help you understand the condition better.

Causes of Bursitis:

Bursitis can be caused by a variety of factors, including:

1. Overuse or repetitive motion: Repeatedly performing the same motion or activity can cause irritation and inflammation in the bursae, leading to bursitis.
2. Trauma or injury: Direct blows or jolts to the affected area can cause bursitis.
3. Age: As we age, the bursae can become less flexible and more prone to inflammation, making older adults more susceptible to bursitis.
4. Medical conditions: Certain medical conditions, such as rheumatoid arthritis, gout, and diabetes, can increase the risk of developing bursitis.
5. Infection: Bacterial or viral infections can cause bursitis by infecting the bursae.

Sym